# Datasets

In this notebook, you will be guided to :
- Search your **project** and describe it
- Search your **mission** and describe it
- Search your **datasets** and describe them
- Display the particularity of vector dataset

#### You may execute this notebook one cell after the other with `Shift + Enter`

For more options, explore `Cell` in the top menu bar, or read a [tutorial](https://www.dataquest.io/blog/jupyter-notebook-tutorial/)

In [1]:
import os
import alteia

In [2]:
import getpass
platform_url = 'https://app.alteia.com'
login = input('Enter your email ')
password = getpass.getpass('Enter your password ')

In [3]:
sdk = alteia.SDK(url=platform_url, user=login, password=password)

# Exercices

### Search your project

In [48]:
company_id = '65a4fcd0586dc233a4f5d8ee'
project_name = '' ### Add your project name
lst_proj = sdk.projects.search(
    filter={
        'company': {'$eq': company_id},
        'name': {'$eq': project_name}
    }
)

### Describe it and search creation date

In [ ]:
my_project = lst_proj[0]
my_project.__dict__
my_project.created

### Search mission in your project

In [ ]:
missions = sdk.missions.search(
  filter={
    'project': {'$eq': my_project.id},
  }
)

for m in missions:
  print(m.name, m.id, m.creation_date)

missions[0].__dict__

### Search datasets in your project / mission

In [ ]:
lst_datasets = sdk.datasets.search(
  filter={
    'project': {'$eq': pid},
    'mission': {'$eq': mid},
  }
)

print(len(lst_datasets))

### List datasets with their name, type, source, categories and ingestio status

In [ ]:
for d in lst_datasets:
  print(d.name, d.id, d.type, d.source, d.categories, d.ingestion['status'])

### Describe the vector dataset

In [ ]:
vct_id = 'vct_id'
vct = sdk.datasets.describe(vct_id)

### Search an annotation

In [ ]:
# Set variable
project_id = "PROJECT_ID"
mission_id = "MISSION_ID"

# At project level
lst_annotations = sdk.annotations.search(
    project=project_id
)

# At mission level
lst_annotations = sdk.annotations.search(
    filter={
        'project': {'$eq': project_id},
        'mission': {'$eq': mission_id}
    }
)